# Installments

In [ ]:
#https://github.com/beir-cellar/beir/blob/main/examples/retrieval/evaluation/dense/evaluate_dpr.py
!pip install beir

# Import Libraries

In [2]:
import numpy as np
import pandas as pd

from time import time
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from sentence_transformers import losses, models, SentenceTransformer
from beir.retrieval.train import TrainRetriever


import logging
import pathlib, os
import random

/usr/local/lib/python3.10/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Connect To Google Drive And Load Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls '/content/drive/MyDrive/project_data'

dpr_models  output_comb  output_norm  scifact


In [5]:
!ls '/content/drive/MyDrive/project_data/scifact'

corpus.jsonl	    queries.jsonl  test.source	   train.csv	    train.target     val.source_rl
process_scifact.py  rephrase	   test.source_rl  train.source     train.target_rl  val.target
qrels		    test.csv	   test.target	   train.source_rl  val.source	     val.target_rl


In [6]:
data_directory = '/content/drive/MyDrive/project_data'

# Load Meta-Data

In [7]:
corpus = pd.read_json(data_directory + '/scifact/corpus.jsonl', lines=True, dtype=str)
corpus

,_id,title,text,metadata
0,4983,Microstructural development of human newborn c...,Alterations of the architecture of cerebral wh...,{}
1,5836,Induction of myelodysplasia by myeloid-derived...,Myelodysplastic syndromes (MDS) are age-depend...,{}
2,7912,"BC1 RNA, the transcript from a master gene for...",ID elements are short interspersed elements (S...,{}
3,18670,The DNA Methylome of Human Peripheral Blood Mo...,DNA methylation plays an important role in bio...,{}
4,19238,The human myelin basic protein gene is include...,Two human Golli (for gene expressed in the oli...,{}
...,...,...,...,...
5178,195689316,Body-mass index and cause-specific mortality i...,BACKGROUND The main associations of body-mass ...,{}
5179,195689757,Targeting metabolic remodeling in glioblastoma...,A key aberrant biological difference between t...,{}
5180,196664003,Signaling architectures that transmit unidirec...,A signaling pathway transmits information from...,{}
5181,198133135,"Association between pre-diabetes, type 2 diabe...",AIMS Trabecular bone score (TBS) is a surrogat...,{}


# Load MyQueries and qrels from .csv files

In [8]:
#train data
df_train = pd.read_csv(data_directory + '/scifact/train.csv', sep='\t', dtype=str)
train_q = df_train[['qid', 'query']]

#test_data
df_test = pd.read_csv(data_directory + '/scifact/test.csv', sep='\t', dtype=str)
test_q = df_test[['qid', 'query']]

test_q

,qid,query
0,1,0-dimensional biomaterials show inductive prop...
1,3,"1,000 genomes project enables mapping of genet..."
2,5,1/2000 in UK have abnormal PrP positivity.
3,13,5% of perinatal mortality is due to low birth ...
4,36,A deficiency of vitamin B12 increases blood le...
...,...,...
334,1379,Women with a higher birth weight are more like...
335,1382,aPKCz causes tumour enhancement by affecting g...
336,1385,cSMAC formation enhances weak ligand signalling.
337,1389,mTORC2 regulates intracellular cysteine levels...


In [9]:
train_q1 = pd.read_csv(data_directory + '/scifact/rephrase/my_train_query1.csv', sep='\t', index_col=0, dtype=str)
train_q2 = pd.read_csv(data_directory + '/scifact/rephrase/my_train_query2.csv', sep='\t', index_col=0, dtype=str)
train_q3 = pd.read_csv(data_directory + '/scifact/rephrase/my_train_query3.csv', sep='\t', index_col=0, dtype=str)
train_q4 = pd.read_csv(data_directory + '/scifact/rephrase/my_train_query4.csv', sep='\t', index_col=0, dtype=str)
train_q5 = pd.read_csv(data_directory + '/scifact/rephrase/my_train_query5.csv', sep='\t', index_col=0, dtype=str)
train_q6 = pd.read_csv(data_directory + '/scifact/rephrase/my_train_query6.csv', sep='\t', index_col=0, dtype=str)

In [25]:
train_q7 = train_q
train_q7['query'] = train_q7['query'] + " " + train_q['query']

train_q8 = train_q
train_q8['query'] = train_q8['query'] + " " + train_q1['query'] + " " + train_q2['query']

In [26]:
#train qrels
df_train_qrels = pd.read_csv(data_directory + '/scifact/qrels/train.tsv', sep='\t', dtype=str)

#test qrels
df_test_qrels = pd.read_csv(data_directory + '/scifact/qrels/test.tsv', sep='\t', dtype=str)

df_test_qrels

,query-id,corpus-id,score
0,1,31715818,1
1,3,14717500,1
2,5,13734012,1
3,13,1606628,1
4,36,5152028,1
...,...,...,...
334,1379,17450673,1
335,1382,17755060,1
336,1385,306006,1
337,1389,23895668,1


# Rewriting data-format to dict

In [27]:
#Note that _id here refers to doc id.
new_corpus = {}
for i in range(len(corpus)):
    key = str(corpus['_id'][i])
    new_corpus[key] = {'text' : corpus['text'][i], 'title' : corpus['title'][i]}

In [28]:
def dict_query(data):
    new_dict = {}
    for i in range(len(data)):
      key = str(data['qid'][i])
      new_dict[key] = data['query'][i]

    return new_dict

In [29]:
dict_tr_q = dict_query(train_q)
dict_tr_q1 = dict_query(train_q1)
dict_tr_q2 = dict_query(train_q2)
dict_tr_q3 = dict_query(train_q3)
dict_tr_q4 = dict_query(train_q4)
dict_tr_q5 = dict_query(train_q5)
dict_tr_q6 = dict_query(train_q6)
dict_tr_q7 = dict_query(train_q7)
dict_tr_q8 = dict_query(train_q8)

dict_te_q = dict_query(test_q)

In [30]:
#train_qrels
new_train_qrels = {}
for i in range(len(df_train_qrels)):
    query_key = str(df_train_qrels['query-id'][i])
    new_train_qrels[query_key] = {str(df_train_qrels['corpus-id'][i]) : int(df_train_qrels['score'][i])}

#test_qrels
new_test_qrels = {}
for i in range(len(df_test_qrels)):
    query_key = str(df_test_qrels['query-id'][i])
    new_test_qrels[query_key] = {str(df_test_qrels['corpus-id'][i]) : int(df_test_qrels['score'][i])}

# Training - Supervised DPR

6 similar strategies are proposed for this 1st approach.

Concatenating 1,2 and 5 rewritten queries together with original query.
Training on a larger data-set. Meaning training is done on 2,3 and 5 separate training data-sets based on rewritten queries.

# Concat and normal strategy

In [31]:
model_name = "msmarco-distilbert-base-v3"

model_norm = SentenceTransformer(model_name)
model_c1 = SentenceTransformer(model_name)
model_c2 = SentenceTransformer(model_name)
model_comb = SentenceTransformer(model_name)


retriever_norm = TrainRetriever(model=model_norm, batch_size=16)
retriever_c1 = TrainRetriever(model=model_c1, batch_size=16)
retriever_c2 = TrainRetriever(model=model_c2, batch_size=16)
retriever_comb = TrainRetriever(model=model_comb, batch_size=16)

train_samples_norm = retriever_norm.load_train(new_corpus, dict_tr_q, new_train_qrels)
train_samples_c1 = retriever_c1.load_train(new_corpus, dict_tr_q7, new_train_qrels)
train_samples_c2 = retriever_c2.load_train(new_corpus, dict_tr_q8, new_train_qrels)
train_samples_comb = retriever_comb.load_train(new_corpus, dict_tr_q6, new_train_qrels)

train_dataloader_norm = retriever_norm.prepare_train(train_samples_norm, shuffle=True)
train_dataloader_c1 = retriever_c1.prepare_train(train_samples_c1, shuffle=True)
train_dataloader_c2 = retriever_c2.prepare_train(train_samples_c2, shuffle=True)
train_dataloader_comb = retriever_comb.prepare_train(train_samples_comb, shuffle=True)

#### Training SBERT with cosine-product
train_loss_norm = losses.MultipleNegativesRankingLoss(model=retriever_norm.model)
train_loss_c1 = losses.MultipleNegativesRankingLoss(model=retriever_c1.model)
train_loss_c2 = losses.MultipleNegativesRankingLoss(model=retriever_c2.model)
train_loss_comb = losses.MultipleNegativesRankingLoss(model=retriever_comb.model)
#### training SBERT with dot-product
#train_loss_norm = losses.MultipleNegativesRankingLoss(model=retriever_norm.model, similarity_fct=util.dot_score)
#train_loss_c1 = losses.MultipleNegativesRankingLoss(model=retriever_c1.model, similarity_fct=util.dot_score)
#train_loss_c2 = losses.MultipleNegativesRankingLoss(model=retriever_c2.model, similarity_fct=util.dot_score)
#train_loss_comb = losses.MultipleNegativesRankingLoss(model=retriever_comb.model, similarity_fct=util.dot_score)

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

Adding Input Examples:   0%|          | 0/51 [00:00<?, ?it/s]

Adding Input Examples:   0%|          | 0/51 [00:00<?, ?it/s]

Adding Input Examples:   0%|          | 0/51 [00:00<?, ?it/s]

Adding Input Examples:   0%|          | 0/51 [00:00<?, ?it/s]

In [32]:
#### Prepare dev evaluator
#ir_evaluator = retriever.load_ir_evaluator(dev_corpus, dev_queries, dev_qrels)

#### If no dev set is present from above use dummy evaluator
ir_evaluator_norm = retriever_norm.load_dummy_evaluator()
ir_evaluator_c1 = retriever_c1.load_dummy_evaluator()
ir_evaluator_c2 = retriever_c2.load_dummy_evaluator()
ir_evaluator_comb = retriever_comb.load_dummy_evaluator()

In [33]:
#### Provide model save path
model_save_path_norm = os.path.join(pathlib.Path(data_directory + '/dpr_models').parent.absolute(), "output_norm", "{}-v1-{}".format(model_name, "SCI-FACT"))
os.makedirs(model_save_path_norm, exist_ok=True)

model_save_path_c1 = os.path.join(pathlib.Path(data_directory + '/dpr_models').parent.absolute(), "output_c1", "{}-v1-{}".format(model_name, "SCI-FACT"))
os.makedirs(model_save_path_c1, exist_ok=True)

model_save_path_c2 = os.path.join(pathlib.Path(data_directory + '/dpr_models').parent.absolute(), "output_c2", "{}-v1-{}".format(model_name, "SCI-FACT"))
os.makedirs(model_save_path_c2, exist_ok=True)

model_save_path_comb = os.path.join(pathlib.Path(data_directory + '/dpr_models').parent.absolute(), "output_comb", "{}-v1-{}".format(model_name, "SCI-FACT"))
os.makedirs(model_save_path_comb, exist_ok=True)

In [38]:
#### Configure Train params
num_epochs = 25
evaluation_steps = 5000
warmup_steps = int(len(train_samples_norm) * num_epochs / retriever_norm.batch_size * 0.1)

In [39]:
retriever_norm.fit(train_objectives=[(train_dataloader_norm, train_loss_norm)],
                evaluator=ir_evaluator_norm,
                epochs=num_epochs,
                output_path=model_save_path_norm,
                warmup_steps=warmup_steps,
                evaluation_steps=evaluation_steps,
                use_amp=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

In [40]:
retriever_c1.fit(train_objectives=[(train_dataloader_c1, train_loss_c1)],
                evaluator=ir_evaluator_c1,
                epochs=num_epochs,
                output_path=model_save_path_c1,
                warmup_steps=warmup_steps,
                evaluation_steps=evaluation_steps,
                use_amp=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

In [41]:
retriever_c2.fit(train_objectives=[(train_dataloader_c2, train_loss_c2)],
                evaluator=ir_evaluator_c2,
                epochs=num_epochs,
                output_path=model_save_path_c2,
                warmup_steps=warmup_steps,
                evaluation_steps=evaluation_steps,
                use_amp=True)

Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

In [42]:
retriever_comb.fit(train_objectives=[(train_dataloader_comb, train_loss_comb)],
                evaluator=ir_evaluator_comb,
                epochs=num_epochs,
                output_path=model_save_path_comb,
                warmup_steps=warmup_steps,
                evaluation_steps=evaluation_steps,
                use_amp=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

# Separate Training - S1

In [43]:
model_name = "msmarco-distilbert-base-v3"

model_s1 = SentenceTransformer(model_name)
retriever_s1 = TrainRetriever(model=model_s1, batch_size=16)




train_samples_s11 = retriever_s1.load_train(new_corpus, dict_tr_q, new_train_qrels)

train_dataloader_s11 = retriever_s1.prepare_train(train_samples_s11, shuffle=True)


#### Training SBERT with cosine-product
train_loss_s11 = losses.MultipleNegativesRankingLoss(model=retriever_s1.model)

#### training SBERT with dot-product
#train_loss_norm = losses.MultipleNegativesRankingLoss(model=retriever_norm.model, similarity_fct=util.dot_score)


ir_evaluator_s11 = retriever_s1.load_dummy_evaluator()

Adding Input Examples:   0%|          | 0/51 [00:00<?, ?it/s]

In [44]:
model_save_path_s1 = os.path.join(pathlib.Path(data_directory + '/dpr_models').parent.absolute(), "output_s1", "{}-v1-{}".format(model_name, "SCI-FACT"))
os.makedirs(model_save_path_s1, exist_ok=True)

In [45]:
retriever_s1.fit(train_objectives=[(train_dataloader_s11, train_loss_s11)],
                evaluator=ir_evaluator_s11,
                epochs=num_epochs,
                output_path=model_save_path_s1,
                warmup_steps=warmup_steps,
                evaluation_steps=evaluation_steps,
                use_amp=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

In [46]:
train_samples_s12 = retriever_s1.load_train(new_corpus, dict_tr_q1, new_train_qrels)

train_dataloader_s12 = retriever_s1.prepare_train(train_samples_s12, shuffle=True)

#### Training SBERT with cosine-product
train_loss_s12 = losses.MultipleNegativesRankingLoss(model=retriever_s1.model)

ir_evaluator_s12 = retriever_s1.load_dummy_evaluator()

Adding Input Examples:   0%|          | 0/51 [00:00<?, ?it/s]

In [47]:
retriever_s1.fit(train_objectives=[(train_dataloader_s12, train_loss_s12)],
                evaluator=ir_evaluator_s12,
                epochs=num_epochs,
                output_path=model_save_path_s1,
                warmup_steps=warmup_steps,
                evaluation_steps=evaluation_steps,
                use_amp=True)

Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

# Separate Training - S2

In [48]:
model_save_path_s2 = os.path.join(pathlib.Path(data_directory + '/dpr_models').parent.absolute(), "output_s2", "{}-v1-{}".format(model_name, "SCI-FACT"))
os.makedirs(model_save_path_s2, exist_ok=True)

In [49]:
train_samples_s2 = retriever_s1.load_train(new_corpus, dict_tr_q2, new_train_qrels)

train_dataloader_s2 = retriever_s1.prepare_train(train_samples_s2, shuffle=True)

train_loss_s2 = losses.MultipleNegativesRankingLoss(model=retriever_s1.model)

ir_evaluator_s2 = retriever_s1.load_dummy_evaluator()

Adding Input Examples:   0%|          | 0/51 [00:00<?, ?it/s]

In [50]:
retriever_s1.fit(train_objectives=[(train_dataloader_s2, train_loss_s2)],
                evaluator=ir_evaluator_s2,
                epochs=num_epochs,
                output_path=model_save_path_s2,
                warmup_steps=warmup_steps,
                evaluation_steps=evaluation_steps,
                use_amp=True)

Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

# Separate Training - S3

In [51]:
model_save_path_s3 = os.path.join(pathlib.Path(data_directory + '/dpr_models').parent.absolute(), "output_s3", "{}-v1-{}".format(model_name, "SCI-FACT"))
os.makedirs(model_save_path_s3, exist_ok=True)

In [52]:
train_samples_s3 = retriever_s1.load_train(new_corpus, dict_tr_q3, new_train_qrels)

train_dataloader_s3 = retriever_s1.prepare_train(train_samples_s3, shuffle=True)

train_loss_s3 = losses.MultipleNegativesRankingLoss(model=retriever_s1.model)

ir_evaluator_s3 = retriever_s1.load_dummy_evaluator()

Adding Input Examples:   0%|          | 0/51 [00:00<?, ?it/s]

In [53]:
retriever_s1.fit(train_objectives=[(train_dataloader_s3, train_loss_s3)],
                evaluator=ir_evaluator_s3,
                epochs=num_epochs,
                output_path=model_save_path_s3,
                warmup_steps=warmup_steps,
                evaluation_steps=evaluation_steps,
                use_amp=True)

Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

# Separate Training - S4

In [54]:
model_save_path_s4 = os.path.join(pathlib.Path(data_directory + '/dpr_models').parent.absolute(), "output_s4", "{}-v1-{}".format(model_name, "SCI-FACT"))
os.makedirs(model_save_path_s4, exist_ok=True)

In [55]:
train_samples_s4 = retriever_s1.load_train(new_corpus, dict_tr_q4, new_train_qrels)

train_dataloader_s4 = retriever_s1.prepare_train(train_samples_s4, shuffle=True)

train_loss_s4 = losses.MultipleNegativesRankingLoss(model=retriever_s1.model)

ir_evaluator_s4 = retriever_s1.load_dummy_evaluator()

Adding Input Examples:   0%|          | 0/51 [00:00<?, ?it/s]

In [56]:
retriever_s1.fit(train_objectives=[(train_dataloader_s4, train_loss_s4)],
                evaluator=ir_evaluator_s4,
                epochs=num_epochs,
                output_path=model_save_path_s4,
                warmup_steps=warmup_steps,
                evaluation_steps=evaluation_steps,
                use_amp=True)

Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

# Separate Training - S5

In [57]:
model_save_path_s5 = os.path.join(pathlib.Path(data_directory + '/dpr_models').parent.absolute(), "output_s5", "{}-v1-{}".format(model_name, "SCI-FACT"))
os.makedirs(model_save_path_s5, exist_ok=True)

In [58]:
train_samples_s5 = retriever_s1.load_train(new_corpus, dict_tr_q5, new_train_qrels)

train_dataloader_s5 = retriever_s1.prepare_train(train_samples_s5, shuffle=True)

train_loss_s5 = losses.MultipleNegativesRankingLoss(model=retriever_s1.model)

ir_evaluator_s5 = retriever_s1.load_dummy_evaluator()

Adding Input Examples:   0%|          | 0/51 [00:00<?, ?it/s]

In [59]:
retriever_s1.fit(train_objectives=[(train_dataloader_s5, train_loss_s5)],
                evaluator=ir_evaluator_s5,
                epochs=num_epochs,
                output_path=model_save_path_s5,
                warmup_steps=warmup_steps,
                evaluation_steps=evaluation_steps,
                use_amp=True)

Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

# Evaluation - Supervised DPR

In [60]:
from beir.retrieval import models

In [61]:
def eval_train_model(model_path, corpus, test_data, qrels):
  evaluator_model = DRES(models.SentenceBERT(model_path, batch_size=16))
  evaluator = EvaluateRetrieval(evaluator_model, score_function="dot")
  results = evaluator.retrieve(corpus, test_data)
  ndcg, _map, recall, precision = evaluator.evaluate(qrels, results, [10,100,1000])
  return ndcg, _map, recall, precision

In [62]:
norm = eval_train_model(model_save_path_norm, new_corpus, dict_te_q, new_test_qrels)
c1 = eval_train_model(model_save_path_c1, new_corpus, dict_te_q, new_test_qrels)
c2 = eval_train_model(model_save_path_c2, new_corpus, dict_te_q, new_test_qrels)
comb = eval_train_model(model_save_path_comb, new_corpus, dict_te_q, new_test_qrels)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

In [67]:
print(norm)
print(c1)
print(c2)
print(comb)

({'NDCG@10': 0.74851, 'NDCG@100': 0.76383, 'NDCG@1000': 0.77031}, {'MAP@10': 0.71088, 'MAP@100': 0.71411, 'MAP@1000': 0.71437}, {'Recall@10': 0.86667, 'Recall@100': 0.93667, 'Recall@1000': 0.98667}, {'P@10': 0.08667, 'P@100': 0.00937, 'P@1000': 0.00099})
({'NDCG@10': 0.72762, 'NDCG@100': 0.74862, 'NDCG@1000': 0.75387}, {'MAP@10': 0.6903, 'MAP@100': 0.69436, 'MAP@1000': 0.69458}, {'Recall@10': 0.84333, 'Recall@100': 0.94333, 'Recall@1000': 0.98333}, {'P@10': 0.08433, 'P@100': 0.00943, 'P@1000': 0.00098})
({'NDCG@10': 0.72952, 'NDCG@100': 0.74711, 'NDCG@1000': 0.75121}, {'MAP@10': 0.68461, 'MAP@100': 0.68848, 'MAP@1000': 0.68861}, {'Recall@10': 0.87, 'Recall@100': 0.95, 'Recall@1000': 0.98333}, {'P@10': 0.087, 'P@100': 0.0095, 'P@1000': 0.00098})
({'NDCG@10': 0.73835, 'NDCG@100': 0.76025, 'NDCG@1000': 0.76439}, {'MAP@10': 0.70438, 'MAP@100': 0.70827, 'MAP@1000': 0.70841}, {'Recall@10': 0.84333, 'Recall@100': 0.95333, 'Recall@1000': 0.98667}, {'P@10': 0.08433, 'P@100': 0.00953, 'P@1000': 

In [64]:
s1 = eval_train_model(model_save_path_s1, new_corpus, dict_te_q, new_test_qrels)
s2 = eval_train_model(model_save_path_s2, new_corpus, dict_te_q, new_test_qrels)
s3 = eval_train_model(model_save_path_s3, new_corpus, dict_te_q, new_test_qrels)
s4 = eval_train_model(model_save_path_s4, new_corpus, dict_te_q, new_test_qrels)
s5 = eval_train_model(model_save_path_s5, new_corpus, dict_te_q, new_test_qrels)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

In [65]:
print(s1)
print(s2)
print(s3)
print(s4)
print(s5)

({'NDCG@10': 0.74219, 'NDCG@100': 0.76374, 'NDCG@1000': 0.76792}, {'MAP@10': 0.70928, 'MAP@100': 0.71375, 'MAP@1000': 0.7139}, {'Recall@10': 0.84667, 'Recall@100': 0.94667, 'Recall@1000': 0.98}, {'P@10': 0.08467, 'P@100': 0.00947, 'P@1000': 0.00098})
({'NDCG@10': 0.75933, 'NDCG@100': 0.77604, 'NDCG@1000': 0.7843}, {'MAP@10': 0.73216, 'MAP@100': 0.73545, 'MAP@1000': 0.7358}, {'Recall@10': 0.84333, 'Recall@100': 0.92333, 'Recall@1000': 0.98667}, {'P@10': 0.08433, 'P@100': 0.00923, 'P@1000': 0.00099})
({'NDCG@10': 0.77625, 'NDCG@100': 0.79172, 'NDCG@1000': 0.79902}, {'MAP@10': 0.75168, 'MAP@100': 0.75415, 'MAP@1000': 0.75444}, {'Recall@10': 0.85333, 'Recall@100': 0.93333, 'Recall@1000': 0.99}, {'P@10': 0.08533, 'P@100': 0.00933, 'P@1000': 0.00099})
({'NDCG@10': 0.75601, 'NDCG@100': 0.7724, 'NDCG@1000': 0.77924}, {'MAP@10': 0.72868, 'MAP@100': 0.73172, 'MAP@1000': 0.73197}, {'Recall@10': 0.84333, 'Recall@100': 0.92333, 'Recall@1000': 0.97667}, {'P@10': 0.08433, 'P@100': 0.00923, 'P@1000': 